In [1]:
import numpy as np
import tensorflow as tf
import cv2 as cv
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.core.framework import tensor_shape_pb2

def export_pb(session):
    with tf.gfile.GFile("ssd_mnv1_const_test_2.pb", "wb") as f:
        inputs = ["image_tensor_float32"] # replace with your input names
        outputs = ["concat", "concat_1"] # replace with your output names
        graph_def = session.graph.as_graph_def(add_shapes=True)
        graph_def = tf.graph_util.convert_variables_to_constants(session, graph_def, outputs)
        graph_def = TransformGraph(
            graph_def,
            inputs,
            outputs,
            [
                "remove_nodes(op=Identity, op=CheckNumerics, op=StopGradient)",
                "sort_by_execution_order", # sort by execution order after each transform to ensure correct node ordering
                "remove_attribute(attribute_name=_XlaSeparateCompiledGradients)",
                "remove_attribute(attribute_name=_XlaCompile)",
                "remove_attribute(attribute_name=_XlaScope)",
                "sort_by_execution_order",
                "remove_device",
                "sort_by_execution_order",
                "fold_batch_norms",
                "sort_by_execution_order",
                "fold_old_batch_norms",
                "sort_by_execution_order"
            ]
        )
        f.write(graph_def.SerializeToString())

# Read the graph.
with tf.gfile.FastGFile('frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    new_model = tf.GraphDef()

old_graph = tf.Graph()
with old_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile('frozen_inference_graph.pb', 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')              

new_model = tf.GraphDef()
with tf.Session(graph=old_graph) as sess:
    for n in sess.graph_def.node:
#         print(n.name)
        if n.name in ['image_tensor']:
            print(n.name,"get.")
#             print(n.op)
#             print(n.attr['T'].type)
            nn = new_model.node.add()
            nn.name = 'image_tensor_float32'
            nn.op = 'Placeholder'
#             nn.attr['T'].CopyFrom(tf.AttrValue(type=tf.float32.as_datatype_enum))
            nn.attr['dtype'].CopyFrom(tf.AttrValue(type=tf.float32.as_datatype_enum))
            dims = [tensor_shape_pb2.TensorShapeProto.Dim(size=1),
                tensor_shape_pb2.TensorShapeProto.Dim(size=300),
                tensor_shape_pb2.TensorShapeProto.Dim(size=300),
                tensor_shape_pb2.TensorShapeProto.Dim(size=3)]
            s = tensor_shape_pb2.TensorShapeProto()
            s.dim.extend(dims)
            nn.attr['shape'].shape.CopyFrom(s)
        elif n.name in ['FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1']:
            print(n.name,"get.")             
            nn = new_model.node.add()    
            nn.op = 'Conv2D'
            nn.name = n.name
            nn.attr['T'].CopyFrom(n.attr['T'])# OK
            nn.attr['data_format'].CopyFrom(tf.AttrValue(s=b'NHWC'))# OK
            nn.attr['dilations'].CopyFrom(n.attr['dilations']) #OK
            nn.attr['padding'].CopyFrom(tf.AttrValue(s=b'SAME')) #OK
            nn.attr['strides'].CopyFrom(tf.AttrValue(list=tf.AttrValue.ListValue(i=[1,2,2,1]))) 
            nn.attr['use_cudnn_on_gpu'].CopyFrom(tf.AttrValue(b = True)) # OK
            
            for i in n.input:
#                 print(name)
                if "Preprocessor/sub".strip()==i.strip():
                    print(i,"====input find.")
                    nn.input.extend(['image_tensor_float32'])
#                     n.input[i] = a.name
                else:
                    nn.input.extend([i])
            
#             for i, name in enumerate(nn.input):
#                 print(name)
        elif n.name in ['ToFloat','detection_boxes','detection_scores','num_detections',
                       'detection_classes','add','add/y','Squeeze','Shape_6',
                       'strided_slice_6','strided_slice_7']:
            ;
        elif n.name.find('Preprocessor')>=0 :
            ;
        elif n.name.find('Postprocessor')>=0 :
            ;
        elif n.name.find('MultipleGridAnchorGenerator')>=0 :
            ;
        else:
            nn = new_model.node.add()
            nn.CopyFrom(n)
    print("remove ok!")

tf.import_graph_def(new_model, name='')
g = tf.get_default_graph()
with g.as_default():
    with tf.Session() as sess:
        export_pb(sess)    
    

/home/cc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: P

Instructions for updating:
Use tf.gfile.GFile.
image_tensor get.
FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1 get.
Preprocessor/sub ====input find.
remove ok!
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
